# 장르별로 가사 크롤링하기! 

### 총 3가지 바꾸셔야합니다.

In [1]:
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

title_artist_lyrics = {'title':[], 'artist':[], 'lyric':[]}
song_num_list = []

# input_url 변경 (1페이지)

In [2]:
input_url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0600"

# 멜론 창 열기
driver = webdriver.Chrome()
driver.get(input_url)

bs = BeautifulSoup(driver.page_source, 'html.parser')
result = bs.find_all('tr')
count = 1

for res in result:
    #print(res)
    # print(f"===== {count} =====")
    # print(res.find('button'))
    if res.find('button'):
        try: 
            # print(f"===== {count} =====")
            song_number = str(res.find('button')).split('-no="')[1][:8]
            song_num_list.append(song_number)
        except: pass
    count +=1

time.sleep(1)
driver.quit()

# input_url 변경 (2페이지)

In [3]:
input_url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0600#params%5BgnrCode%5D=GN0600&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex=51"
# 멜론 창 열기
driver = webdriver.Chrome()
driver.get(input_url)
bs = BeautifulSoup(driver.page_source, 'html.parser')
result = bs.find_all('tr')
count = 1
for res in result:
    #print(res)
    # print(f"===== {count} =====")
    # print(res.find('button'))
    if res.find('button'):
        try: 
            # print(f"===== {count} =====")
            song_number = str(res.find('button')).split('-no="')[1][:8]
            song_num_list.append(song_number)
        except: pass
    count +=1

time.sleep(1)
driver.quit()

In [4]:
print(len(song_num_list))

50


In [5]:
title_artist_lyrics = {'title':[], 'artist':[], 'lyric':[]}

url_front = 'https://www.melon.com/song/detail.htm?songId='
for num in song_num_list:
    url = url_front + num
    driver = webdriver.Chrome()
    driver.get(url)
    try:
        # 가사가 로드될 때까지 대기
        lyric_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'lyric'))
        )
        
        # text 부분만 뽑아서 print
        lyric_text = lyric_element.text
        # print(lyric_text)
        
        # 노래 제목 뽑기
        title_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))
        title_text = title_element.text
        # print(f"노래 제목 : {title_text}")
        
        # 아티스트 뽑기
        artist_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'artist_name')))
        artist_text = artist_element.text
        # print(f"가수 이름 : {artist_text}")
        
        title_artist_lyrics['title'].append(title_text)
        title_artist_lyrics['artist'].append(artist_text)
        title_artist_lyrics['lyric'].append(lyric_text)
        print('dictionary에 추가함')
    except Exception as e: print("실패", e)

    driver.quit()
    
# print(title_artist_lyrics)

dictionary에 추가함
dictionary에 추가함
dictionary에 추가함
dictionary에 추가함
dictionary에 추가함
dictionary에 추가함
dictionary에 추가함
dictionary에 추가함
dictionary에 추가함
실패 Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF68EE870C2+63090]
	(No symbol) [0x00007FF68EDF2D12]
	(No symbol) [0x00007FF68EC8EC65]
	(No symbol) [0x00007FF68ECD499D]
	(No symbol) [0x00007FF68ECD4ADC]
	(No symbol) [0x00007FF68ED15B37]
	(No symbol) [0x00007FF68ECF701F]
	(No symbol) [0x00007FF68ED13412]
	(No symbol) [0x00007FF68ECF6D83]
	(No symbol) [0x00007FF68ECC83A8]
	(No symbol) [0x00007FF68ECC9441]
	GetHandleVerifier [0x00007FF68F28262D+4238301]
	GetHandleVerifier [0x00007FF68F2BF78D+4488509]
	GetHandleVerifier [0x00007FF68F2B7A6F+4456479]
	GetHandleVerifier [0x00007FF68EF60606+953270]
	(No symbol) [0x00007FF68EDFE5DF]
	(No symbol) [0x00007FF68EDF92B4]
	(No symbol) [0x00007FF68EDF93EB]
	(No symbol) [0x00007FF68EDE9C24]
	BaseThreadInitThunk [0x00007FFC07C3257D+29]
	RtlUserThreadStart [0x00007FFC092AAA58+40]
dictionary에 추가함
dictionary에 추

In [6]:
len(title_artist_lyrics['lyric'])

46

In [7]:
df_song = pd.DataFrame(title_artist_lyrics)
df_song.head()

,title,artist,lyric
0,12,오스터존 (OSTERJON),못 받은 전화 미안하지만\n새하얀 세상과 난 어울리지 않아\n난 괜찮은데 왜 그런 ...
1,개화,청슬 (ChungSeul),우연히 찾아온 너란 큰 세상 속에\n두 눈을 감고서 온전히 널 느껴\n\n쓰린 아픔...
2,흐르는 대로,Adios Audio,흐르는 대로 드리운 대로 느끼는 대로\n\n지독하리 찌는 기분\n흩어지지 못하는 개...
3,돼지와 함께 춤을,리얼프로그,나의 현실은 쓰라리곤 아팠고\n우린 쏟아져내렸어\n구멍 난 하루의 나의 모든 파란을...
4,Horizon (수평선),Vanillare (바닐레어),Just set me free\nFrom all the messes you’re i...


# 파일 저장명 변경

In [8]:
save_name = 'rock.csv'
df_song.to_csv(save_name)
print('csv 파일로 저장 완료')

csv 파일로 저장 완료
